In [ ]:
from python_simulation_manager.handler import ExperimentHandler
from python_simulation_manager.output import ExperimentOutput
from typing import override
from pathlib import Path
import numpy as np

In [ ]:
class C_Output(ExperimentOutput):
    def __init__(self, out_path):
        super().__init__(out_path)
        self.title   = None
        self.result1 = []
        self.result2 = []

    @override
    def parse_output(self, line_number, line):
        if line_number == 0:
            self.title = line.strip()
            return
        
        slines = line.split("=>")
        if line_number == 1:
            self.result1.append(float(slines[1]))
        else:
            self.result2.append(float(slines[1]))
        
            
class C_Experiment(ExperimentHandler):
    
    def lengths(self) -> list[int]:
        return self.get_scale_variables()

In [ ]:
class Builder:
    
    def __init__(self, result_folder: str, experiment_name: str):
        self.proj_dir        = Path.cwd().parent
        self.result_folder   = Path(result_folder)
        self.experiment_name = experiment_name
        
    def build(self, monte_carlo_trials: dict[int, int], temperature: np.ndarray) -> C_Experiment:
        executable  = self.proj_dir / "c_example" / "build" / "c_example.exe"
        
        writer      = C_Experiment(self.proj_dir, self.experiment_name, self.result_folder, verbose_log=False)
        
        writer.set_output_type(C_Output)
        writer.add_static_parameter("temperature", temperature)
        writer.add_scaling_parameter("monte_carlo_trials", monte_carlo_trials)
        writer.set_scale_variable_names(["length"])        
        writer.set_executable(executable)
        writer.set_files()

        return writer
    
    def load(self, lengths: list[int]) -> C_Experiment:
        proj_dir        = Path.cwd().parent
        exp             = C_Experiment(proj_dir, self.experiment_name, self.result_folder, verbose_log=False)
        
        exp.set_output_type(C_Output)
        exp.set_scale_variables(lengths)
        exp.set_scale_variable_names(["length"])
        exp.set_files()
        return exp

In [ ]:
result_folder = "test_folder"
exp_folder    = "c_experiment"

builder = Builder(result_folder, exp_folder)

monte_carlo_trials = {
    2: 1_000,
    3: 5_000,
    4: 10_000
}
temperature = np.arange(0, 10, 1, np.int32)
experiment  = builder.build(monte_carlo_trials, temperature)
experiment.write_parameter_files()

In [ ]:
experiment.are_parameter_files_available()

In [ ]:
for length in experiment.lengths():
    experiment.run_executable(length, verbose=False)

In [ ]:
experiment.are_results_available()

In [ ]:
results = experiment.get_results()

In [ ]:
for (length, res) in results.items():
    res: C_Output # ""Downcast""" result from Parent class (ExperimentOutPut) to child class "C_output", for type hintint
    
    print(f"For length = {length}:")
    print(f"> title: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")
    
    assert type(res.result1) == np.ndarray, "Lists are cast to np array automatically!"
    assert type(res.result2) == np.ndarray, "Lists are cast to np array automatically!"

From file

In [ ]:
result_folder = "test_folder"
exp_folder    = "c_experiment"
lengths       = [2,3,4]

exp_loader     = Builder(result_folder, exp_folder)
exp_from_file  = exp_loader.load(lengths) 

In [ ]:
exp_from_file.are_results_available()

In [ ]:
results_from_file = exp_from_file.get_results()

In [ ]:
for (length, res) in results.items():
    res: C_Output
    
    print(f"For length = {length}:")
    print(f"> title: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")